# Challenge 2: ETL Pipeline - Data Transformation

## Task Description
In this challenge, we need to:
1. Apply transformations to extracted data
2. Handle data quality issues
3. Optimize transformation operations
4. Prepare data for loading phase

## Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Create a Spark session
spark = SparkSession.builder \
    .appName("ETL Pipeline - Data Transformation") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "8") \
    .getOrCreate()

# Set log level
spark.sparkContext.setLogLevel("WARN")

## Load Data from Challenge 1

Let's assume we have the DataFrames from Challenge 1. If not, we'll recreate them:

In [ ]:
# PostgreSQL connection parameters
jdbc_url = "jdbc:postgresql://postgres:5432/datamart"
connection_properties = {
    "user": "spark",
    "password": "spark",
    "driver": "org.postgresql.Driver"
}

# Load tables
customers_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "raw.customers") \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .load()

orders_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "raw.orders") \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .load()

order_items_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "raw.order_items") \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .load()

products_df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "raw.products") \
    .option("user", connection_properties["user"]) \
    .option("password", connection_properties["password"]) \
    .option("driver", connection_properties["driver"]) \
    .load()

## Data Cleaning

In [ ]:
# TODO: Handle null values in customer data
clean_customers_df = customers_df \
    .filter(col("email").isNotNull()) \
    .withColumn("first_name", when(col("first_name").isNull(), "Unknown").otherwise(col("first_name"))) \
    .withColumn("last_name", when(col("last_name").isNull(), "Unknown").otherwise(col("last_name"))) \
    .withColumn("full_name", concat_ws(" ", col("first_name"), col("last_name")))

# Check results
clean_customers_df.show(5)

## Data Standardization

In [ ]:
# TODO: Standardize data formats
standardized_orders_df = orders_df \
    .withColumn("order_date", to_date(col("order_date"))) \
    .withColumn("order_year", year(col("order_date"))) \
    .withColumn("order_month", month(col("order_date"))) \
    .withColumn("order_day", dayofmonth(col("order_date"))) \
    .withColumn("status", upper(col("status"))) \
    .withColumn("is_completed", when(col("status") == "COMPLETED", True).otherwise(False))

# Check results
standardized_orders_df.show(5)

## Data Enrichment

In [ ]:
# TODO: Join datasets to create enriched view
# Calculate order totals and join with customer information

# First calculate order totals
order_totals_df = order_items_df \
    .groupBy("order_id") \
    .agg(
        sum(col("quantity") * col("price")).alias("order_total"),
        count("*").alias("num_items")
    )

# Join orders with totals
orders_with_totals = standardized_orders_df \
    .join(order_totals_df, "order_id")

# Join with customer data
enriched_orders_df = orders_with_totals \
    .join(clean_customers_df, "customer_id")

# Select final columns
final_enriched_df = enriched_orders_df.select(
    "order_id",
    "customer_id",
    "full_name",
    "email",
    "order_date",
    "order_year",
    "order_month",
    "status",
    "order_total",
    "num_items"
)

# Check results
final_enriched_df.show(5)

## Advanced Transformation: Customer Segmentation

In [ ]:
# TODO: Implement customer segmentation logic
# Calculate metrics like total spend, frequency, recency

# Aggregate metrics by customer
customer_metrics = final_enriched_df \
    .groupBy("customer_id", "full_name", "email") \
    .agg(
        sum("order_total").alias("total_spend"),
        count("*").alias("order_count"),
        max("order_date").alias("last_order_date")
    )

# Calculate recency in days
customer_metrics = customer_metrics \
    .withColumn("recency_days", datediff(current_date(), col("last_order_date")))

# Assign segments
customer_segments = customer_metrics \
    .withColumn("spend_segment", 
               when(col("total_spend") > 1000, "High")
               .when(col("total_spend") > 500, "Medium")
               .otherwise("Low")) \
    .withColumn("frequency_segment", 
               when(col("order_count") > 10, "High")
               .when(col("order_count") > 5, "Medium")
               .otherwise("Low")) \
    .withColumn("recency_segment", 
               when(col("recency_days") < 30, "Active")
               .when(col("recency_days") < 90, "Recent")
               .otherwise("Inactive"))

# Create overall segment
customer_segments = customer_segments \
    .withColumn("customer_segment", 
               when((col("spend_segment") == "High") & (col("recency_segment") == "Active"), "VIP")
               .when((col("spend_segment") == "High") | (col("frequency_segment") == "High"), "Premium")
               .when((col("recency_segment") == "Inactive"), "At Risk")
               .otherwise("Regular"))

# Check results
customer_segments.show(5)

## Optimized Transformations

In [ ]:
# TODO: Optimize transformations using caching, partitioning, etc.

# Cache frequently used DataFrames
clean_customers_df.cache()
standardized_orders_df.cache()

# Repartition for better parallelism
num_partitions = 8
repartitioned_orders = standardized_orders_df.repartition(num_partitions, "customer_id")
repartitioned_customers = clean_customers_df.repartition(num_partitions, "customer_id")

# Perform join with optimized partitioning
optimized_join = repartitioned_orders \
    .join(repartitioned_customers, "customer_id")

# Check partition count
print(f"Original orders partitions: {standardized_orders_df.rdd.getNumPartitions()}")
print(f"Repartitioned orders: {repartitioned_orders.rdd.getNumPartitions()}")
print(f"Optimized join partitions: {optimized_join.rdd.getNumPartitions()}")

## Handling Schema Evolution

In [ ]:
# TODO: Implement schema evolution handling

def add_missing_columns(df, required_columns):
    """Add any missing columns to the DataFrame with null values"""
    current_columns = df.columns
    for column in required_columns:
        if column not in current_columns:
            df = df.withColumn(column, lit(None))
    return df

def select_required_columns(df, required_columns):
    """Select only the required columns in the specified order"""
    # First ensure all required columns exist
    df_with_all_columns = add_missing_columns(df, required_columns)
    # Then select only required columns in correct order
    return df_with_all_columns.select(*required_columns)

# Example usage
required_order_columns = ["order_id", "customer_id", "order_date", "status", "total", "tax", "shipping"]

# standardized_orders_with_schema_handling = select_required_columns(standardized_orders_df, required_order_columns)
# standardized_orders_with_schema_handling.show(5)